In [ ]:
!git clone https://github.com/sunsmarterjie/yolov12.git

Cloning into 'yolov12'...
remote: Enumerating objects: 1159, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 1159 (delta 324), reused 296 (delta 296), pack-reused 680 (from 2)
Receiving objects: 100% (1159/1159), 1.81 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (580/580), done.


In [ ]:
cd yolov12

/content/yolov12


In [ ]:
!wget wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl

--2025-08-22 07:44:04--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-08-22 07:44:04--  https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/494232964/b7fb7022-424c-4ac2-b46b-a41f386edaa5?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-22T08%3A24%3A15Z&rscd=attachment%3B+filename%3Dflash_attn-2.7.3%2Bcu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-22T07%3A24%3A14Z&ske=2025-08-22T08%3A24%3A15Z&sks=b&skv=2018-11-09&sig=zQHKzIJi1H%2F3NXb8ea

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 98.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.0 MB/s eta 0:00:00


In [ ]:
!unzip prepared_dataset_detection.zip

Archive:  prepared_dataset_detection.zip
   creating: prepared_dataset_detection/
  inflating: __MACOSX/._prepared_dataset_detection  
  inflating: prepared_dataset_detection/.DS_Store  
  inflating: __MACOSX/prepared_dataset_detection/._.DS_Store  
   creating: prepared_dataset_detection/images/
  inflating: __MACOSX/prepared_dataset_detection/._images  
   creating: prepared_dataset_detection/labels/
  inflating: __MACOSX/prepared_dataset_detection/._labels  
  inflating: prepared_dataset_detection/data.yaml  
  inflating: __MACOSX/prepared_dataset_detection/._data.yaml  
  inflating: prepared_dataset_detection/images/.DS_Store  
  inflating: __MACOSX/prepared_dataset_detection/images/._.DS_Store  
   creating: prepared_dataset_detection/images/train/
  inflating: __MACOSX/prepared_dataset_detection/images/._train  
   creating: prepared_dataset_detection/images/val/
  inflating: __MACOSX/prepared_dataset_detection/images/._val  
  inflating: prepared_dataset_detection/labels/.DS_Sto

In [ ]:
#!/usr/bin/env python3
"""
YOLOv12 trainer (Colab / CUDA, T4 GPU)
- Uses CUDA if available, else CPU.
- Safe defaults for a small single-class dataset.
"""

import os
from pathlib import Path
import torch
from ultralytics import YOLO

# ---- CUDA-friendly settings (not MPS) ----
torch.backends.cudnn.benchmark = True         # speed up convs with fixed input size
# (don't set MPS/float32 matmul tweaks on CUDA)

# Paths
DATA_YAML = Path("/content/yolov12/prepared_dataset_detection/data.yaml").resolve()

def main():
    assert DATA_YAML.exists(), f"data.yaml not found at {DATA_YAML}"
    device = "0" if torch.cuda.is_available() else "cpu"  # "0" selects first CUDA GPU (T4)
    n_workers = min(8, os.cpu_count() or 2)

    # Start small (n). If VRAM is comfy, try 'yolov12s.pt' later.
    model = YOLO("yolov12n.pt")

    results = model.train(
        data=str(DATA_YAML),
        epochs=300,              # you were still learning at 100; early-stop will prevent overrun
        batch=32,                # T4 (16GB) should handle 32 at 640 for 'n'; drop to 16 if OOM
        imgsz=640,               # try 960 later for recall (reduce batch if you do)
        device=device,           # "0" for CUDA GPU, "cpu" fallback
        amp=True,                # mixed precision is stable on CUDA; faster and less VRAM
        optimizer="SGD",         # make lr0/momentum obeyed (auto would override)
        lr0=0.01,                # classic YOLO LR; good starting point with SGD
        momentum=0.937,
        weight_decay=5e-4,
        cos_lr=True,             # smoother schedule, usually helpful
        patience=50,             # early stopping on val plateau
        workers=n_workers,       # dataloader workers
        cache=True,              # cache labels+images to RAM for faster epochs in Colab
        project="runs",
        name="pothole_yolov12_t4",
        save=True,
        # If you want modest augs once stable, uncomment:
        mosaic=0.5,         # default is 1.0, 0.5 is more gentle
        mixup=0.05,         # light blend
        copy_paste=0.1,     # occasional copy-paste
        fliplr=0.5,         # random horizontal flip
        translate=0.1,
        scale=0.5,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4
        )

    # Evaluate on val split
    metrics = model.val()
    print(metrics)

if __name__ == "__main__":
    main()

New https://pypi.org/project/ultralytics/8.3.183 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=/content/yolov12/prepared_dataset_detection/data.yaml, epochs=300, time=None, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=2, project=runs, name=pothole_yolov12_t43, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning /content/yolov12/prepared_dataset_detection/labels/train.cache... 238 images, 23 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.3GB RAM): 100%|██████████| 238/238 [00:00<00:00, 380.38it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all         42         40      0.914      0.675      0.741      0.563
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/pothole_yolov12_t43
Ultralytics 8.3.63 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 376 layers, 2,508,539 parameters, 0 gradients, 5.8 GFLOPs


val: Scanning /content/yolov12/prepared_dataset_detection/labels/val.cache... 42 images, 7 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.0GB RAM): 100%|██████████| 42/42 [00:00<00:00, 342.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.55it/s]


                   all         42         40      0.899      0.664      0.728      0.544
Speed: 0.2ms preprocess, 8.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/pothole_yolov12_t432
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c73ac48b320>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,  

In [ ]:
import shutil
import os

source_folder = '/content/yolov12/runs/pothole_yolov12_t43'
destination_folder = '/content/drive/MyDrive/pothole_yolov12_t43'

if os.path.exists('/content/drive'):
  shutil.copytree(source_folder, destination_folder)
  print(f"Folder '{source_folder}' copied to '{destination_folder}'")
else:
  print("Google Drive is not mounted. Please mount Google Drive and try again.")

Folder '/content/yolov12/runs/pothole_yolov12_t43' copied to '/content/drive/MyDrive/pothole_yolov12_t43'
